# Наивный байесовский классификатор для классификации спам-сообщений

In [1]:
import numpy as np
import pandas as pd

Прочитаем файл (разделителем здесь выступает символ табуляции).

In [2]:
df = pd.read_csv(
    "data/SMSSpamCollection.csv", header=None, sep="\t", names=["Label", "SMS"]
)

df.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Посмотрим, сколько объектов каждого класса присутствует в датасете.

In [3]:
df["Label"].value_counts()

ham     4825
spam     747
Name: Label, dtype: int64

In [4]:
df["Label"].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

Мы видим, что датасет несбалансированный – не-спама значительно больше, чем спама.

# Предобработка данных
Удаляем символы, не являющиеся буквами, приводим тексты SMS к нижнему регистру, разбиваем строки на слова.

In [5]:
df["SMS"] = (
    df["SMS"].str.replace(r"\W+", " ", regex=True).str.lower().str.split()
)

In [6]:
df.head()

,Label,SMS
0,ham,"[go, until, jurong, point, crazy, available, o..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,..."


# Разделение на обучающую и тестовую выборки
Разобьём датасет на обучающую и тестовую выборку в соотношении 80/20.Так как датасет несбалансированный, необходимо провести стратификацию – в обучающей и тестовой выборке должна быть примерно одна и та же доля спама.

In [7]:
df_train = df.groupby("Label", group_keys=False).apply(
    lambda x: x.sample(frac=0.8)
)

df_test = df.drop(df_train.index)

In [8]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [9]:
df_train["Label"].value_counts(normalize=True)

ham     0.865859
spam    0.134141
Name: Label, dtype: float64

In [10]:
df_test["Label"].value_counts(normalize=True)

ham     0.866248
spam    0.133752
Name: Label, dtype: float64

Мы видим, что и в обучающей, и в тестовой выборке содержится примерно 86.5% спама – как и в нашем оригинальном датасете.

In [11]:
print(f"{df_train.shape=:}")
print(f"{df_test.shape=:}")

df_train.shape=(4458, 2)
df_test.shape=(1114, 2)


In [12]:
df_test.head()

,Label,SMS
0,ham,"[nah, i, don, t, think, he, goes, to, usf, he,..."
1,ham,"[i, m, gonna, be, home, soon, and, i, don, t, ..."
2,ham,"[ahhh, work, i, vaguely, remember, that, what,..."
3,spam,"[07732584351, rodger, burns, msg, we, tried, t..."
4,ham,"[great, i, hope, you, like, your, man, well, e..."


# Список слов
Создаём список всех слов, встречающихся в обучающей выборке.

In [13]:
vocabulary = list(set(df_train["SMS"].sum()))

In [14]:
vocabulary[11:20]

['doin',
 'chatting',
 'info',
 'jelly',
 'interfued',
 'dammit',
 'crazyin',
 'cruise',
 'wales']

In [15]:
len(vocabulary)

7792

# Встречаемость слов
Для каждого SMS-сообщения посчитаем, сколько раз в нём встречается каждое слово.

In [16]:
words_counts = pd.DataFrame(
    [
        [text.count(word) for word in vocabulary]
        for text in df_train["SMS"].items()
    ],
    columns=vocabulary,
)

Добавим частоты каждого слова в обучающий датасет.

In [17]:
df_train = pd.concat([df_train, words_counts], axis=1)

In [18]:
df_train.head()

,Label,SMS,tired,mouth,attach,motivate,ec2a,offered,plans,turkeys,...,golden,advance,aint,hockey,married,dude,bedroom,organise,raj,extract
0,ham,"[i, m, leaving, my, house, now]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[from, tomorrow, onwards, eve, 6, to, 3, work]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[discussed, with, your, mother, ah]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[i, m, in, town, now, so, i, ll, jus, take, mr...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[ok, i, found, dis, pierre, cardin, one, which...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Значения для формулы Байеса
Посчитаем необходимые значения для формулы Байеса.

In [19]:
alpha = 1

In [20]:
n_vocabulary = len(vocabulary)

In [21]:
p_ham, p_spam = df_train["Label"].value_counts(normalize=True)

In [22]:
n_spam = df_train[df_train["Label"] == "spam"]["SMS"].apply(len).sum()

In [23]:
n_ham = df_train[df_train["Label"] == "ham"]["SMS"].apply(len).sum()

In [24]:
print(f"{n_vocabulary=:}")
print(f"{p_spam=:}")
print(f"{p_ham=:}")
print(f"{n_spam=:}")
print(f"{n_ham=:}")

n_vocabulary=7792
p_spam=0.1341408703454464
p_ham=0.8658591296545536
n_spam=15198
n_ham=57058


# Формула Байеса
Напишем функцию для формулы Байеса.

In [25]:
def p_word_spam(word: str) -> float:
    """
    По формуле Байеса считает вероятность того, что слово является спамом.

    Args:
        word: слово

    Returns:
        Вероятность того, что слово – спам (1, если слова нет в df_train).
    """
    return (
        (df_train[df_train["Label"] == "spam"][word].sum() + alpha)
        / (n_spam + alpha * n_vocabulary)
        if word in vocabulary
        else 1.0
    )

# Алгоритм классификации
Напишем функцию для классификации сообщения.

In [26]:
def classify(message) -> str:
    """
    Классифицирует сообщение как спам или не спам.

    Args:
        message: сообщение (список слов)

    Returns:
        "spam" или "ham"
    """
    p_spam_message = p_spam * np.prod(
        [p_word_spam(word) for word in message]
    )

    p_ham_message = p_ham * np.prod(
        [1 - p_word_spam(word) for word in message]
    )

    return "spam" if p_spam_message < p_ham_message else "ham"

# Предсказания
Сделаем предсказания на тестовых данных. Это может занять несколько минут.

In [27]:
df_test["Prediction"] = [
    classify(message) for message in df_test["SMS"].tolist()
]

In [28]:
df_test.head()

,Label,SMS,Prediction
0,ham,"[nah, i, don, t, think, he, goes, to, usf, he,...",spam
1,ham,"[i, m, gonna, be, home, soon, and, i, don, t, ...",ham
2,ham,"[ahhh, work, i, vaguely, remember, that, what,...",ham
3,spam,"[07732584351, rodger, burns, msg, we, tried, t...",ham
4,ham,"[great, i, hope, you, like, your, man, well, e...",ham


Посмотрим, какой процент сообщений в тестовой выборке наш алгоритм классифицировал правильно.

In [29]:
accuracy = (df_test["Prediction"] == df_test["Label"]).sum() / len(df_test)
print(f"Правильных предсказаний {accuracy * 100:3f} %")

Правильных предсказаний 40.484740 %


Посмотрим на сообщения, которые наш алгоритм классифицировал неправильно.

In [30]:
df_test[df_test["Prediction"] != df_test["Label"]].head()

,Label,SMS,Prediction
0,ham,"[nah, i, don, t, think, he, goes, to, usf, he,...",spam
3,spam,"[07732584351, rodger, burns, msg, we, tried, t...",ham
6,ham,"[tell, where, you, reached]",spam
8,ham,"[k, did, you, call, me, just, now, ah]",spam
9,ham,"[i, m, really, not, up, to, it, still, tonight...",spam


# Наивный байесовский классификатор в sklearn
Ура, мы реализовали наивный байесовский классификатор с нуля!
А теперь посмотрим, как то же самое можно сделать с помощью библиотеки scikit-learn.

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import ComplementNB

Прочитаем заново csv-файл и предобработаем данные. Разбивать сообщения на слова в этот раз не нужно.

In [32]:
df = pd.read_csv(
    "data/SMSSpamCollection.csv", header=None, sep="\t", names=["Label", "SMS"]
)

In [33]:
df["SMS"] = df["SMS"].str.replace(r"\W+", " ", regex=True).str.lower()

Преобразуем строки в векторный вид – то есть, снова создадим таблицу с частотами слов. Но в этот раз уже не вручную.

In [34]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["SMS"])

Определим целевую переменную.

In [35]:
y = df["Label"]

С помощью функции `train_test_split` из scikit-learn разобьём выборку на обучающую и тестовую в пропорции 80/20. Не забудем сделать стратификацию!

In [36]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y
)

Обучаем наивный байесовский классификатор. Параметр `alpha` у него по умолчанию равен 1.

In [37]:
clf = ComplementNB()

In [38]:
clf.fit(X_train, y_train)

ComplementNB()

In [39]:
y_pred = clf.predict(X_test)

In [40]:
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.9730941704035875


Как мы видим, классификатор из sklearn работает намного лучше.